In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [1]:
from src.interface import TrainerModule
TrainerModule.main(stage = 0 , resume = 0 , checkname= 1)


24-05-23 19:26:58|MOD:display     |: Model Specifics:
24-05-23 19:26:58|MOD:display     |: Start Process [Data] at Thu May 23 19:26:58 2024!


--Process Queue : Data + Fit + Test
--Model_name is set to gru_ShortTest!
Callback : DynamicDataLink() , assign and unlink dynamic data in tra networks
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=10) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=10) , display batch progress bar
Callback : StatusDisplay(verbosity=10) , display epoch and event information
{'random_seed': None,
 'model_name': 'gru_ShortTest',
 'model_module': 'gru

24-05-23 19:27:00|MOD:display     |: Finish Process [Data], Cost 1.6 Secs
24-05-23 19:27:00|MOD:display     |: Start Process [Fit] at Thu May 23 19:27:00 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


  0%|          | 0/10 [00:00<?, ?it/s]

score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


Valid Ep#  0 score : 0.01745: 100%|██████████| 10/10 [00:01<00:00,  8.88it/s]
FirstBite Ep#  0 : loss  1.01026, train-0.00940, valid-0.05371, best-0.0537, lr1.3e-03
Valid Ep#  1 score : -0.12253: 100%|██████████| 10/10 [00:01<00:00,  8.58it/s]
FirstBite Ep#  1 : loss  0.98937, train 0.01660, valid 0.03397, best 0.0340, lr2.5e-03
Valid Ep#  2 score : 0.02425: 100%|██████████| 10/10 [00:01<00:00,  8.77it/s]
FirstBite Ep#  2 : loss  0.96003, train 0.03806, valid 0.11886, best 0.1189, lr3.8e-03
Valid Ep#  3 score : 0.05437: 100%|██████████| 10/10 [00:01<00:00,  8.58it/s]
FirstBite Ep#  3 : loss  0.94099, train 0.06354, valid 0.07503, best 0.1189, lr5.0e-03
24-05-23 19:28:09|MOD:display     |: gru_ShortTest #0 @20170103|FirstBite Ep#  4 Max Epoch|Train 0.0000 Valid 0.0000 BestVal 0.1189|Cost  1.1Min, 13.5Sec/Ep
24-05-23 19:28:09|MOD:display     |: Finish Process [Fit], Cost 0.0 Hours, 1.2 Min/model, 17.4 Sec/Epoch
24-05-23 19:28:09|MOD:display     |: Start Process [Test] at Thu May 23 19:28

                    hook_name  num_calls  total_time   avg_time
10             on_train_batch         40   37.830830   0.945771
21           on_fit_model_end          1   25.120991  25.120991
27              on_test_batch        105   16.542808   0.157551
15        on_validation_batch         40    4.536801   0.113420
4          on_fit_model_start          1    2.056511   2.056511
24        on_test_model_start          1    0.569816   0.569816
17    on_validation_epoch_end          4    0.046753   0.011688
25   on_test_model_type_start          3    0.045765   0.015255
12         on_train_epoch_end          4    0.010536   0.002634
29     on_test_model_type_end          3    0.004022   0.001341
13  on_validation_epoch_start          4    0.000998   0.000250
3                on_fit_start          1    0.000998   0.000998
6        on_train_epoch_start          4    0.000997   0.000249
23              on_test_start          1    0.000000   0.000000
0          on_configure_model          1

In [3]:
import torch
s1 = torch.load('./model/gru_lgbm_ShortTest/0/20170103/best/lgbm_string.pt')

In [4]:
s2 = torch.load('./lgbm_string.pt')

In [6]:
s2 == s1

True

In [ ]:
# prepare data
from src.interface import NetDataModule
NetDataModule.prepare_data()